In [4]:
import pandas as pd
import numpy as np

# Sample Data Creation
data = {
    "Date": pd.date_range(start="2023-01-01", periods=20, freq='D'),
    "Product Name": np.random.choice(['Product A', 'Product B', 'Product C', 'Product D'], 20),
    "Units Sold": np.random.randint(1, 10, size=20),
    "Revenue": np.random.randint(100, 1000, size=20),
    "Region": np.random.choice(['North', 'South', 'East', 'West'], 20),
    "Discount Offered (%)": np.random.randint(0, 30, size=20),
    "Salesperson": np.random.choice(['Alice', 'Bob', 'Charlie', 'David'], 20)
}

df = pd.DataFrame(data)

# Calculate Actual Revenue After Discount
df['Actual Revenue'] = df['Revenue'] * (1 - df['Discount Offered (%)'] / 100)

# 1. Top 3 sales transactions with the highest revenue
top_3_revenue = df.nlargest(3, 'Revenue')
print(top_3_revenue)

# 2. Units of each product sold
units_sold = df.groupby('Product Name')['Units Sold'].sum()
print(units_sold)

# 3. Total revenue after applying discounts
total_revenue_after_discount = df['Actual Revenue'].sum()
print(total_revenue_after_discount)

# 4. Transaction with the highest discount offered
max_discount_transaction = df.loc[df['Discount Offered (%)'].idxmax()]
print(max_discount_transaction[['Discount Offered (%)', 'Actual Revenue']])

# 5. Salesperson generating the highest total revenue
highest_revenue_salesperson = df.groupby('Salesperson')['Actual Revenue'].sum().idxmax()
print(highest_revenue_salesperson)

# 6. Average discount offered by each salesperson
avg_discount_per_salesperson = df.groupby('Salesperson')['Discount Offered (%)'].mean()
print(avg_discount_per_salesperson)

# 7. Revenue generated in each region
revenue_by_region = df.groupby('Region')['Actual Revenue'].sum()
print(revenue_by_region)

# 8. Region where Alice generated the highest sales
alice_sales = df[df['Salesperson'] == 'Alice']
highest_sales_region = alice_sales.groupby('Region')['Actual Revenue'].sum().idxmax()
print(highest_sales_region)

# 9. Product generating the highest revenue per unit sold
df['Revenue Per Unit'] = df['Revenue'] / df['Units Sold']
highest_revenue_product = df.groupby('Product Name')['Revenue Per Unit'].mean().idxmax()
print(highest_revenue_product)

# 10. Transactions rated as "High" performance
high_performance_threshold = 700  # Define threshold for high performance
high_performance_count = df[df['Revenue'] > high_performance_threshold].shape[0]
print(high_performance_count)

# 11. Salesperson selling most units in North region without discounts
north_sales = df[(df['Region'] == 'North') & (df['Discount Offered (%)'] == 0)]

if not north_sales.empty:
    best_north_salesperson = north_sales.groupby('Salesperson')['Units Sold'].sum().idxmax()
    print(best_north_salesperson)
else:
    print("No sales found in the North region with zero discount.")

# 12. Average revenue per unit sold in each region for each product
avg_revenue_per_unit = df.groupby(['Region', 'Product Name']).apply(lambda x: (x['Revenue'].sum() / x['Units Sold'].sum())).reset_index(name='Avg Revenue Per Unit')
print(avg_revenue_per_unit)

# 13. Salesperson with highest average revenue after discounts
avg_revenue_per_salesperson = df.groupby('Salesperson')['Actual Revenue'].mean()
highest_avg_salesperson = avg_revenue_per_salesperson.idxmax()
print(highest_avg_salesperson)

# 14. Cumulative total revenue over time for each salesperson
df['Date'] = pd.to_datetime(df['Date'])
cumulative_revenue = df.groupby(['Salesperson', 'Date'])['Actual Revenue'].sum().groupby(level=0).cumsum().reset_index()

# Merge the cumulative revenue back into the original DataFrame
df = df.merge(cumulative_revenue, on=['Salesperson', 'Date'], how='left', suffixes=('', '_Cumulative'))
df.rename(columns={'Actual Revenue_Cumulative': 'Cumulative Revenue'}, inplace=True)

print(df[['Salesperson', 'Date', 'Cumulative Revenue']])

# 15. Top 2 transactions for each salesperson
top_2_transactions = df.groupby('Salesperson').apply(lambda x: x.nlargest(2, 'Revenue'))
print(top_2_transactions)

# 16. Cumulative revenue generated by each product over time
daily_cumulative_revenue = df.groupby(['Date', 'Product Name'])['Actual Revenue'].sum().cumsum().reset_index()
print(daily_cumulative_revenue)

# 17. Analyze how discounts affect revenue
revenue_discount_analysis = df.groupby('Product Name').agg(
    Revenue_With_Discount=('Actual Revenue', lambda x: x[df['Discount Offered (%)'] > 0].mean()),
    Revenue_Without_Discount=('Actual Revenue', lambda x: x[df['Discount Offered (%)'] == 0].mean())
)
print(revenue_discount_analysis)

# 18. Weighted average discount offered by each salesperson
weighted_avg_discount = df.groupby('Salesperson').apply(lambda x: np.average(x['Discount Offered (%)'], weights=x['Actual Revenue']))
print(weighted_avg_discount)

# 19. Percentage of total revenue by region
total_revenue = df['Actual Revenue'].sum()
revenue_percentage = df.groupby('Region')['Actual Revenue'].sum() / total_revenue * 100
print(revenue_percentage)


         Date Product Name  Units Sold  Revenue Region  Discount Offered (%)  \
16 2023-01-17    Product D           3      973   West                    18   
9  2023-01-10    Product C           3      900   West                     7   
8  2023-01-09    Product B           5      862   East                    22   

   Salesperson  Actual Revenue  
16         Bob          797.86  
9      Charlie          837.00  
8          Bob          672.36  
Product Name
Product A    11
Product B    16
Product C    41
Product D    13
Name: Units Sold, dtype: int32
8274.029999999999
Discount Offered (%)        29
Actual Revenue          105.08
Name: 13, dtype: object
Charlie
Salesperson
Alice      22.000000
Bob        20.142857
Charlie    11.250000
David      28.500000
Name: Discount Offered (%), dtype: float64
Region
East     1754.23
North    3504.68
West     3015.12
Name: Actual Revenue, dtype: float64
North
Product C
5
No sales found in the North region with zero discount.
  Region Product Nam